In [1]:
import pandas as pd
import sys
sys.path.append("..")
import pandas as pd
from pathlib import Path
import numpy as np
import pickle
from sif_src.glove import load_glove_vectors, calculate_glove_corpus_embeddings
from sif_src.sif import flatten, compute_sif_embeddings_queries, compute_sif_embeddings_texts, compute_word_frequencies, compute_sif_weights, remove_pc_sif
import tqdm
from collections import defaultdict, Counter
from collections.abc import Iterable
from sklearn.decomposition import TruncatedSVD


Calculate and pickle both embeddings for training set

In [2]:
train_df = pd.read_pickle("../pickle_backups/marco_train_df2024-06-04T17.38.1717490321.pickle")
train_passages = train_df["passages"]
train_texts = [passage["passage_text"] for passage in train_passages]
train_df["texts"] = train_texts

#homogenize MS MARCO columns
train_df['query'] = train_df['query'].apply(lambda x: [x])

# Load GloVe vectors
glove_vectors = load_glove_vectors('../wv/glove.6B.50d.txt')


In [3]:
queries = train_df["query"]
texts = train_df["texts"]
word_freq = compute_word_frequencies(queries + texts)

compute SIF embeddings

In [4]:
sif_weights = compute_sif_weights(word_freq)

In [5]:
query_sif_embeddings = compute_sif_embeddings_queries(queries, glove_vectors, sif_weights)
texts_sif_embeddings = compute_sif_embeddings_texts(texts, glove_vectors, sif_weights)

In [6]:
texts_sif_embeddings = remove_pc_sif(texts_sif_embeddings, alpha=0.001)

In [7]:
train_df["sif_queries"] = list(query_sif_embeddings)
train_df["sif_texts"] = list(texts_sif_embeddings)

In [8]:
#queries = train_df["query"]
#texts = train_df["texts"]

query_glove_embeddings = calculate_glove_corpus_embeddings(queries, glove_vectors)


In [9]:
texts_glove_embeddings = calculate_glove_corpus_embeddings(texts, glove_vectors)

In [10]:
train_df["glove_queries"] = list(query_glove_embeddings)
train_df["glove_texts"] = list(texts_glove_embeddings)

In [11]:
import pickle
from datetime import datetime

TIMESTAMP = datetime.now()
OUTPUT_PATH = Path("../pickle_backups")
TIMESTAMP_STR = TIMESTAMP.isoformat().replace(":", ".")

with open(OUTPUT_PATH / 
    f"train_embeddings{TIMESTAMP.strftime(r'%Y-%m-%dT%H.%M.%s')}.pickle",
    "wb"
) as f:
    pickle.dump(train_df, f)

In [12]:
train_df

,answers,passages,query,query_id,query_type,wellFormedAnswers,texts,sif_queries,sif_texts,glove_queries,glove_texts
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",[what is rba],19699,description,[],"[Since 2007, the RBA's outstanding reputation ...","[-0.5499105637254654, -0.7749932194904489, -0....","[[-0.06280740234170455, -0.163671952027229, 0....","[[0.17393333713213602, -0.024219670643409092, ...","[[0.4105771131251218, 0.09751141514137106, 0.0..."
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",[was ronald reagan a democrat],19700,description,[],"[In his younger years, Ronald Reagan was a mem...","[-0.4944724931295951, 0.3329963899260163, 0.65...","[[-0.15682234241979096, 0.053772786060203207, ...","[[-0.2430683970451355, 0.24003800451755525, 0....","[[0.00016572205133216325, 0.23466255689083143,..."
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",[how long do you need for sydney and surroundi...,19701,numeric,[],"[Sydney, New South Wales, Australia is located...","[-0.5686715681183611, 0.7886818234882632, -0.7...","[[-0.08781571105351091, -0.1970642659003113, -...","[[0.3590648201527074, 0.3001724988222122, -0.0...","[[0.3626662904911853, 0.28700647249500805, -0...."
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",[price to install tile in shower],19702,numeric,[],"[In regards to tile installation costs, consum...","[0.3390108882129254, 0.2936569853751151, 0.292...","[[0.17516424218913831, 0.10020178445042328, -0...","[[0.3093609996140003, 0.11179117175439994, 0.3...","[[0.3843396766291511, 0.31139479567777484, 0.2..."
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",[why conversion observed in body],19703,description,[],"[Conclusions: In adult body CT, dose to an org...","[0.49261088639952527, 0.030099666190201834, -0...","[[-0.058887265109216734, 0.20776162665288703, ...","[[0.42426599860191344, 0.0575822040438652, -0....","[[0.4383610147130439, 0.11237455838186255, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...
891052,[255077833],"{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]...",[nasa credit union routing number],462353,NUMERIC,[255077833 is NASA's credit union routing numb...,[Complete a Direct Dispute Form if you have in...,"[1.545546277559988, 0.5569648552434452, 1.3301...","[[0.13836077699551172, 0.009487162116366932, -...","[[0.3137059897184372, 0.2028579980134964, 0.70...","[[0.4284991400404456, 0.10947319016968077, 0.1..."
891053,[Galveston County],"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",[what county is hitchcock texas located in],607295,LOCATION,[],[Most of the city of Hitchcock is served by th...,"[0.17882708201675537, 0.29605803235660705, -0....","[[-0.0993255512907615, -0.052689596415482126, ...","[[0.07771856231348855, 0.3966410058949675, -0....","[[0.35306263440533686, 0.30042826325485583, -0..."
891054,[All about great blowout style for about thirt...,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",[what is blowout for hair],724819,DESCRIPTION,[],"[And with no waiting period, the treatment is ...","[0.4116498186812494, -0.6613837877992375, 0.97...","[[-0.009333419992647399, 0.1174666330464707, -...","[[0.2347540020942688, 0.1842821903526783, -0.1...","[[0.25159782473929226, 0.15846253686198375, -0..."
891055,[No Answer Present.],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 0], '...",[what car does isaac bell drive in the wrecker],583161,ENTITY,[],[Isaac Bell is the son of Ebenezer Bell and gr...,"[-0.6717660963294712, -0.37379412170352916, 0....","[[-0.012309304533940356, 0.1300399202227089, -...","[[0.08927911147475243, 0.06939421966671944, 0....","[[0.32903673350811, 0.47709633310635885, -0.21..."
